<a href="https://colab.research.google.com/github/maxysio/DS-PRJ2-DataStructures/blob/master/05-BlockChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import hashlib
from time import gmtime, strftime
import time

In [0]:
class Block:
    def __init__(self, timestamp, data, previous_hash, current_hash):
      self.timestamp = timestamp
      self.data = data
      self.previous_hash = previous_hash
      self.hash = current_hash

In [0]:
class BlockChain:
  def __init__(self):
    self.blocks = {}
    self.head = None
    self.tail = None

  def calc_hash(data):
    sha = hashlib.sha256()
    sha.update(data.encode('utf-8'))
    return sha.hexdigest()

  def prepend(self, data):
    # Create a new block
    data_hash = self.calc_hash(data)
    new_block = Block(time.gmtime(), data, None, data_hash)
    self.blocks.append(new_block)

    if self.head is None:
      # If no head, that would mean no tail, assign both to new block and return
      self.head = new_block
      self.tail = new_block
      return
    
    # Assign the new blocks hash to the previous hash of the head and move head to the new block
    self.head.previous_hash = new_block.hash
    self.head = new_block

  def append(self, data):
    # Create a new block
    data_hash = self.calc_hash(data)
    new_block = Block(time.gmtime(), data, None, data_hash)
    self.blocks.append(new_block)

    if self.head is None:
      # If no head, that would mean no tail, assign both to new block and return
      self.head = new_block
      self.tail = new_block
      return

    # Assign the hash from the tail to the new blocks previous hash, move the tail to the new block
    new_block.previous_hash = self.tail.hash
    self.tail = new_block


  def search(self, data):
    # Traverse through the list and find based on data or hash
    # Or via head/tail
    # ?????
    
  def remove(self, data):
    pass

  def pop(self):
    pass

  def insert(self, data):
    pass

  def size(self):
    pass